In [1]:
from src.value import Value
import src.function as F
import numpy as np

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X, y = load_breast_cancer(return_X_y = True)
X_train, X_test, y_train, y_test = train_test_split(X, y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = Value(X_train, "x_train", requires_grad=False)
X_test = Value(X_test, "x_test", requires_grad=False)
y_train = Value(y_train, "y_train", requires_grad=False)
y_test = Value(y_test, "y_test", requires_grad=False)

In [3]:
w = Value(np.random.randn(X.shape[1]) * 10**(-2), "w", requires_grad = True)
b = Value([0.0], "b", requires_grad = True)

In [4]:
lr = 0.01

matmul1 = F._FunctionFactory().get_new_function_of_type(F._Matmul)
add1 = F._FunctionFactory().get_new_function_of_type(F._Add)
loss = F._FunctionFactory().get_new_function_of_type(F._BCELossWithLogits)
    
def fit(X, num_epoch):     
    for i in range(num_epoch):
        
        y = add1.forward(matmul1.forward(X, w), b)
        l = loss.forward(y, y_train)
        loss.backward()
        w.value -= lr * w.grad
        b.value -= lr * b.grad
        l.zero_grad()   
        
        # if i % 10:
        print(f"Epoch {i+1}/{num_epoch}: Train Loss: {l.value}")

def predict(X):
    y = add1.forward(matmul1.forward(X, w), b)
    y_pred = F.sigmoid(y).value > 0.5
    
    return y_pred

Creating Function of type <class 'src.function._Matmul'> with name matmul_0
Creating Function of type <class 'src.function._Add'> with name add_0
Creating Function of type <class 'src.function._BCELossWithLogits'> with name bce_loss_logit_0


In [5]:
fit(X_train, 100)    

Epoch 1/100: Train Loss: 0.7051148382969286
Epoch 2/100: Train Loss: 0.3472942175188646
Epoch 3/100: Train Loss: 0.17740595717207752
Epoch 4/100: Train Loss: 0.10905738619447392
Epoch 5/100: Train Loss: 0.09655378235925857
Epoch 6/100: Train Loss: 0.08884531424639686
Epoch 7/100: Train Loss: 0.08418212799123365
Epoch 8/100: Train Loss: 0.08106300885517237
Epoch 9/100: Train Loss: 0.07874253561456618
Epoch 10/100: Train Loss: 0.07689010185825322
Epoch 11/100: Train Loss: 0.07533915467005417
Epoch 12/100: Train Loss: 0.07399643063486572
Epoch 13/100: Train Loss: 0.07280619625894955
Epoch 14/100: Train Loss: 0.07173337431692178
Epoch 15/100: Train Loss: 0.07075467286585571
Epoch 16/100: Train Loss: 0.0698537605819889
Epoch 17/100: Train Loss: 0.06901861441210733
Epoch 18/100: Train Loss: 0.06824003056308256
Epoch 19/100: Train Loss: 0.06751075390334563
Epoch 20/100: Train Loss: 0.0668249398871715
Epoch 21/100: Train Loss: 0.06617780165825228
Epoch 22/100: Train Loss: 0.06556536599592865
E

In [6]:
y_train_pred = predict(X_train)
y_test_pred = predict(X_test)

Creating Function of type <class 'src.function._Sigmoid'> with name sigmoid_0
Creating Function of type <class 'src.function._Sigmoid'> with name sigmoid_1


In [7]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, auc, roc_curve, matthews_corrcoef

def print_metrics(y_true, y_pred):
    conf_matrix = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = conf_matrix.ravel()
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)    

    print(f"Confusion Matrix")
    print(f"{conf_matrix}\n")
    print(f"Accuracy: {(tp + tn)/(tp + tn + fn + fp)}")
    print(f"Precision: {tp / (tp + fp)}")
    print(f"Recall: {tp / (tp + fn)}")
    print(f"AUC score: {auc(fpr, tpr)}")
    print(f"MCC score: {mcc}")
    print(f"F1-Score: {(tp)/(tp+(fp+fn)/2)}\n")
    
print_metrics(y_train.value, y_train_pred)
print("")
print_metrics(y_test.value, y_test_pred)

Confusion Matrix
[[156   4]
 [  0 266]]

Accuracy: 0.9906103286384976
Precision: 0.9851851851851852
Recall: 1.0
AUC score: 0.9874999999999999
MCC score: 0.9800793618659437
F1-Score: 0.9925373134328358


Confusion Matrix
[[51  1]
 [ 2 89]]

Accuracy: 0.9790209790209791
Precision: 0.9888888888888889
Recall: 0.978021978021978
AUC score: 0.9793956043956045
MCC score: 0.9549644877199515
F1-Score: 0.9834254143646409

